In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# SK하이닉스 종목코드
ticker = "127980"

# 시작 날짜와 종료 날짜 설정
start_date = "2020-03-20"
end_date = "2024-03-20"
# 주어진 기간 동안의 일별 거래량 정보 가져오기
df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """MACD 및 MACD 신호 계산"""
    df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    return df

def calculate_rsi(df, window=14, signal_window=9):
    """RSI 및 RSI 신호 계산"""
    delta = df['종가'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # RSI 신호선 추가
    df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
    return df

# MACD 및 MACD 신호 계산
df = calculate_macd(df)

df = calculate_rsi(df)

# '종가' 컬럼이 포함된 DataFrame을 가정합니다. 예를 들어, df라고 합시다.
# df = pd.read_csv('path_to_your_data.csv') # 데이터 파일을 로드하는 예시

# 지정된 기간에 대한 SMA 계산
periods = [5, 20, 60, 120, 240]
for period in periods:
    df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

# 지정된 기간에 대한 EMA 계산
for period in periods:
    df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

for i in range(len(periods)):
    for j in range(i + 1, len(periods)):
        smaller_period = periods[i]
        larger_period = periods[j]
        df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

# EMA 간의 차이 계산
for i in range(len(periods)):
    for j in range(i + 1, len(periods)):
        smaller_period = periods[i]
        larger_period = periods[j]
        df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
    df[f'{column}_등락률'] = df[column].pct_change() * 100

# SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
periods = [5, 20, 60, 120, 240]

# SMA 등락률 계산 및 추가
for period in periods:
    df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

# EMA 등락률 계산 및 추가
for period in periods:
    df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

# 'MACD'가 0 이상일 때 1, 아니면 0을 할당
df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

# 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

# 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

# 시가 대비 종가 등락률 컬럼 추가
df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
# 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
for period in [5, 20, 60, 120, 240]:
    df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

# 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
for period in [5, 20, 60, 120, 240]:
    df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


# csv_file_path = "C:/apps/h1/이스트소프트_data.csv"  # 저장할 파일 경로 및 이름 설정
# df.to_csv(csv_file_path, encoding='utf-8-sig')

#print(f"Data saved to {csv_file_path}")

# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = df['등락률'].shift(-1)
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df1= df.iloc[300:, :]
X = df.drop(['next_day_return','target'], axis=1)
y = df['target']
# # 무한대 값 NaN으로 변환
# X.replace([np.inf, -np.inf], np.nan, inplace=True)

# # NaN 값이 포함된 행 제거
# X.dropna(inplace=True)

# # 대상 변수 y도 동일한 행 제거를 위해 X와 인덱스를 맞춥니다.
# y = y.reindex(X.index)

# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

# Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# # Combining all the data into a single dataframe without merging by feature
# final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
#                       features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)

# csv_file_path = "C:/apps/h1/NH프라임리츠_importance_data.csv"  # 저장할 파일 경로 및 이름 설정
# final_df.to_csv(csv_file_path, encoding='utf-8-sig')
top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
X_top8 = X[top_8_features]
y = df['target'].map({-1: 0, 1: 1})  # y 값을 0과 1로 조정
X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=42)


# 모델 초기화
models = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
}

# 점수를 저장할 딕셔너리
scores = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': []
}

# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores['Model'].append(model_name)
    scores['Accuracy'].append(accuracy_score(y_test, y_pred))
    scores['Precision'].append(precision_score(y_test, y_pred, average='macro'))
    scores['Recall'].append(recall_score(y_test, y_pred, average='macro'))
    scores['F1_Score'].append(f1_score(y_test, y_pred, average='macro'))

# 점수 딕셔너리를 데이터프레임으로 변환
scores_df = pd.DataFrame(scores)
scores_df

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy,Precision,Recall,F1_Score
0,SVC,0.428571,0.214286,0.500,0.300000
1,LogisticRegression,1.000000,1.000000,1.000,1.000000
2,DecisionTreeClassifier,1.000000,1.000000,1.000,1.000000
3,RandomForestClassifier,0.857143,0.875000,0.875,0.857143


In [2]:
correlations.head(10)

,Feature,Correlation
0,종가_minus_SMA_240,0.625418
1,EMA_240_등락률,0.615501
2,종가_minus_EMA_240,0.615103
3,EMA_5_minus_EMA_120,0.581526
4,종가_minus_SMA_120,0.570395
5,종가_minus_EMA_120,0.541792
6,EMA_120_등락률,0.537313
7,SMA_120_등락률,0.511327
8,EMA_5_minus_EMA_240,0.506694
9,종가_minus_SMA_20,0.468876


In [3]:
df

,시가,고가,저가,종가,거래량,등락률,EMA_short,EMA_long,MACD,MACD_Signal,...,종가_minus_SMA_60,종가_minus_SMA_120,종가_minus_SMA_240,종가_minus_EMA_5,종가_minus_EMA_20,종가_minus_EMA_60,종가_minus_EMA_120,종가_minus_EMA_240,next_day_return,target
날짜,,,,,,,,,,,,,,,,,,,,,
2024-02-08,8690,8700,8510,8550,30323,0.470035,8570.032325,8498.825141,71.207185,126.831300,...,369.333333,165.333333,-2511.500000,29.690967,9.213190,140.018437,-529.831277,-2575.895842,1.754386,1
2024-02-13,8680,8900,8650,8700,63022,1.754386,8590.027352,8513.726982,76.300370,116.725114,...,505.000000,330.833333,-2318.125000,119.793978,144.050029,280.509636,-373.553074,-2405.763926,2.413793,1
2024-02-14,8570,8910,8540,8910,49920,2.413793,8639.253913,8543.080539,96.173375,112.614766,...,700.500000,548.750000,-2070.500000,219.862652,320.330979,474.427352,-160.849718,-2177.541819,-1.683502,-1
2024-02-15,8910,8960,8750,8760,46618,-1.683502,8657.830234,8559.148647,98.681587,109.828130,...,539.166667,407.583333,-2189.916667,46.575101,154.108981,313.790390,-305.711706,-2308.226119,-1.598174,-1
2024-02-16,8770,8800,8610,8620,27263,-1.598174,8652.010198,8563.656155,88.354044,105.533313,...,387.000000,276.000000,-2300.416667,-62.283266,12.765268,168.092344,-438.344570,-2427.908890,2.320186,1
2024-02-19,8600,8910,8600,8820,44828,2.320186,8677.854783,8582.644588,95.210196,103.468690,...,574.000000,482.916667,-2077.333333,91.811156,192.501909,356.023743,-234.404991,-2209.420020,-0.793651,-1
2024-02-20,8830,8850,8660,8750,20023,-0.793651,8688.954047,8595.041285,93.912762,101.557504,...,493.666667,421.750000,-2125.375000,14.540771,110.835061,276.645915,-299.373503,-2260.503671,1.257143,1
2024-02-21,8760,8920,8640,8860,39971,1.257143,8715.268809,8614.667856,100.600953,101.366194,...,589.833333,537.083333,-1995.208333,83.027181,199.803150,373.969000,-186.243363,-2132.657168,-2.483070,-1
2024-02-22,8890,9030,8640,8640,42999,-2.483070,8703.688992,8616.544311,87.144681,98.521891,...,357.666667,324.666667,-2194.708333,-91.315213,-18.273340,148.920836,-399.528596,-2333.133042,-1.504630,-1
